In [1]:
from tqdm.notebook import trange, tqdm
import os

import numpy as np
import pandas as pd
import swifter

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

os.chdir("..")
from src.utils import BETA_STR
from src.config import Ns, betas

In [2]:
def ingest_noise_data(path):
    df_list = []
    files = os.listdir(path)
    for i in range(len(files)):
        noise = float(os.path.splitext(files[i])[0].split("=")[1])
        filename = os.path.join("data/ib_to_decay/", files[i])
        data = pd.read_csv(filename)
        data = data.rename_axis(index="N", columns="beta_ind").melt(ignore_index=False, value_name="lambda").reset_index()
        data["beta_ind"] = data["beta_ind"].astype(np.int32)
        data["noise"] = noise
        df_list.append(data)
    return pd.concat(df_list, ignore_index=True)

In [3]:
data = ingest_noise_data("data/ib_to_decay/")

In [6]:
def plot_beta_vs_lambda(data, n=None):
    title = f"Lambda Regularization vs {BETA_STR}"
    if n is None:
        df_to_plot = data.groupby(["beta_ind", "noise"], as_index=False)["lambda"].mean()
        title += " (halflives are averaged)"
    else:
        df_to_plot = data.loc[data["N"]==n]
        title += f" N={n}"

    df_to_plot["betas"] = df_to_plot["beta_ind"].apply(lambda i: betas[i])
    fig = px.line(df_to_plot, x="betas", y="lambda", color="noise", markers=True, 
                  title=title,
                 labels = {"betas":BETA_STR})
    return fig

In [8]:
fig = plot_beta_vs_lambda(data, n=None)
fig.show()

In [10]:
fig.write_html("../plots/lambda_vs_beta.html")